<a href="https://colab.research.google.com/github/yohanesnuwara/66DaysOfData/blob/main/D10_FuzzyStringMatching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fuzzy String Matching

Someone posting an interesting problem in his Twitter that a dataset contains multiple spelling issues of "Philadelphia", such as "Philadelpia" or "Phila". The task is to fix these data. One suggests using Levenshtein distance to determine how similar two words are. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Install FuzzyWuzzy
!pip install -q fuzzywuzzy
from fuzzywuzzy import fuzz, process

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
!wget https://github.com/yohanesnuwara/datasets/raw/master/PPP_data_150k_plus.csv.rar
!unrar x "/content/PPP_data_150k_plus.csv.rar" "/content"

--2021-07-13 03:52:09--  https://github.com/yohanesnuwara/datasets/raw/master/PPP_data_150k_plus.csv.rar
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/yohanesnuwara/datasets/master/PPP_data_150k_plus.csv.rar [following]
--2021-07-13 03:52:09--  https://raw.githubusercontent.com/yohanesnuwara/datasets/master/PPP_data_150k_plus.csv.rar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18167052 (17M) [application/octet-stream]
Saving to: ‘PPP_data_150k_plus.csv.rar’

PPP_data_150k_plus. 100%[===================>]  17.33M   112MB/s    in 0.2s    

2021-07-13 03:52:10 (112 MB/s) - ‘PPP_data_150k_plus

In [3]:
df = pd.read_csv("/content/PPP_data_150k_plus.csv/PPP_data_150k_plus.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 661218 entries, 0 to 661217
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   LoanRange      661218 non-null  object 
 1   BusinessName   661210 non-null  object 
 2   Address        661201 non-null  object 
 3   City           661203 non-null  object 
 4   State          661218 non-null  object 
 5   Zip            661202 non-null  float64
 6   NAICSCode      654435 non-null  float64
 7   BusinessType   659789 non-null  object 
 8   RaceEthnicity  661218 non-null  object 
 9   Gender         661218 non-null  object 
 10  Veteran        661218 non-null  object 
 11  NonProfit      42462 non-null   object 
 12  JobsRetained   620712 non-null  float64
 13  DateApproved   661218 non-null  object 
 14  Lender         661218 non-null  object 
 15  CD             661218 non-null  object 
dtypes: float64(3), object(13)
memory usage: 80.7+ MB


Masking cities starting with "PH"



In [4]:
cities = df.City.unique().astype(str)

cities_p = [c for c in cities if c.startswith("PH")]
cities_p

['PHENIX CITY',
 'PHIL CAMPBELL',
 'PHENIX',
 'PHOENIX',
 'PHOENIXA',
 'PHOENIX,',
 'PHOENX',
 'PHOENIX AZ 85017',
 'PHONEIX',
 'PHEONIX',
 'PHILO',
 'PHELAN',
 'PHILLIPSBURG',
 'PHILPOT',
 'PHELPS',
 'PHILLIPSTON',
 'PHIPPSBURG',
 'PHILADELPHIA',
 'PHEBA',
 'PHILIPSBURG',
 'PHILMONT',
 'PHILIPSTOWN',
 'PHOENICIA',
 'PHILOMATH',
 'PHOENIXVILLE',
 'PHILA',
 'PHILDADELPHIA',
 'PHILADELPHIA,',
 'PHIALDELPHIA',
 'PHIADELPHIA',
 'PHILADELPIA',
 'PHILADELKPHIA',
 'PHOENIXVILE',
 'PHILA.',
 'PHILADELPHILA',
 'PHILADEPHIA',
 'PHILDELPHIA',
 'PHILIP',
 'PHARR',
 'PHILLIPS',
 'PHILIPPI']

Now, not all cities that starts with "PH" are not actually "PHILADELPHIA". For example, "PHILLIPSBURG" Therefore, we need to look at cities in US as reference. Look another dataset.

In [5]:
!wget https://raw.githubusercontent.com/yohanesnuwara/datasets/master/us_cities.csv

city_df = pd.read_csv('/content/us_cities.csv')

city_df.head()

--2021-07-13 03:52:14--  https://raw.githubusercontent.com/yohanesnuwara/datasets/master/us_cities.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1720210 (1.6M) [text/plain]
Saving to: ‘us_cities.csv’

us_cities.csv       100%[===================>]   1.64M  --.-KB/s    in 0.08s   

2021-07-13 03:52:14 (19.4 MB/s) - ‘us_cities.csv’ saved [1720210/1720210]



,ID,STATE_CODE,STATE_NAME,CITY,COUNTY,LATITUDE,LONGITUDE
0,1,AK,Alaska,Adak,Aleutians West,55.999722,-161.207778
1,2,AK,Alaska,Akiachak,Bethel,60.891854,-161.392330
2,3,AK,Alaska,Akiak,Bethel,60.890632,-161.199325
3,4,AK,Alaska,Akutan,Aleutians East,54.143012,-165.785368
4,5,AK,Alaska,Alakanuk,Kusilvak,62.746967,-164.602280


These are all cities that starts with "PH".

In [6]:
cities = city_df.CITY.unique().astype(str)

cities_actual = [c.upper() for c in cities if c.upper().startswith("PH")]
cities_actual

['PHENIX CITY',
 'PHIL CAMPBELL',
 'PHOENIX',
 'PHELAN',
 'PHILLIPSVILLE',
 'PHILO',
 'PHIPPSBURG',
 'PHILLIPSBURG',
 'PHELPS',
 'PHILPOT',
 'PHYLLIS',
 'PHILLIPS',
 'PHILADELPHIA',
 'PHEBA',
 'PHILIPP',
 'PHILIPSBURG',
 'PHILLIPSPORT',
 'PHILMONT',
 'PHOENICIA',
 'PHILOMATH',
 'PHOENIXVILLE',
 'PHILIP',
 'PHARR',
 'PHENIX',
 'PHILOMONT',
 'PHLOX',
 'PHILIPPI']

Masking out the cities list, we have name of cities starting with "PH" that are mispelled or invalid names. Most of them referring to "PHILADELPHIA".

In [7]:
ph_incorrect = [c for c in cities_p if c not in cities_actual]

ph_incorrect

['PHOENIXA',
 'PHOENIX,',
 'PHOENX',
 'PHOENIX AZ 85017',
 'PHONEIX',
 'PHEONIX',
 'PHILLIPSTON',
 'PHILIPSTOWN',
 'PHILA',
 'PHILDADELPHIA',
 'PHILADELPHIA,',
 'PHIALDELPHIA',
 'PHIADELPHIA',
 'PHILADELPIA',
 'PHILADELKPHIA',
 'PHOENIXVILE',
 'PHILA.',
 'PHILADELPHILA',
 'PHILADEPHIA',
 'PHILDELPHIA']

Use FuzzyWuzzy to calculate similarity between two words. A name from the above list, for example, "PHILADELKPHIA", is compared to every city name in the `cities_actual` list. We can see "PHILADELKPHIA" and "PHILADELPHIA" has similarity score 96. 

In [8]:
example = ph_incorrect[14]

for potential in cities_actual:
    print(f"{example} -> {potential} = {fuzz.ratio(example, potential)}")

PHILADELKPHIA -> PHENIX CITY = 33
PHILADELKPHIA -> PHIL CAMPBELL = 54
PHILADELKPHIA -> PHOENIX = 30
PHILADELKPHIA -> PHELAN = 42
PHILADELKPHIA -> PHILLIPSVILLE = 38
PHILADELKPHIA -> PHILO = 44
PHILADELKPHIA -> PHIPPSBURG = 35
PHILADELKPHIA -> PHILLIPSBURG = 48
PHILADELKPHIA -> PHELPS = 53
PHILADELKPHIA -> PHILPOT = 50
PHILADELKPHIA -> PHYLLIS = 30
PHILADELKPHIA -> PHILLIPS = 57
PHILADELKPHIA -> PHILADELPHIA = 96
PHILADELKPHIA -> PHEBA = 33
PHILADELKPHIA -> PHILIPP = 50
PHILADELKPHIA -> PHILIPSBURG = 42
PHILADELKPHIA -> PHILLIPSPORT = 48
PHILADELKPHIA -> PHILMONT = 38
PHILADELKPHIA -> PHOENICIA = 45
PHILADELKPHIA -> PHILOMATH = 55
PHILADELKPHIA -> PHOENIXVILLE = 40
PHILADELKPHIA -> PHILIP = 53
PHILADELKPHIA -> PHARR = 33
PHILADELKPHIA -> PHENIX = 32
PHILADELKPHIA -> PHILOMONT = 36
PHILADELKPHIA -> PHLOX = 33
PHILADELKPHIA -> PHILIPPI = 57


In [9]:
best_match = process.extractOne(example, cities_actual)
print(best_match)

('PHILADELPHIA', 96)


Now, for each name in our incorrect name list, determine which one actual city name has the best match.  

In [10]:
for c in ph_incorrect:
  best_city_name = process.extractOne(c, cities_actual)[0]
  print(f"{c} -> {best_city_name}")

PHOENIXA -> PHOENIX
PHOENIX, -> PHOENIX
PHOENX -> PHOENIX
PHOENIX AZ 85017 -> PHOENIX
PHONEIX -> PHOENIX
PHEONIX -> PHENIX
PHILLIPSTON -> PHILLIPS
PHILIPSTOWN -> PHILIP
PHILA -> PHILADELPHIA
PHILDADELPHIA -> PHILADELPHIA
PHILADELPHIA, -> PHILADELPHIA
PHIALDELPHIA -> PHILADELPHIA
PHIADELPHIA -> PHILADELPHIA
PHILADELPIA -> PHILADELPHIA
PHILADELKPHIA -> PHILADELPHIA
PHOENIXVILE -> PHOENIXVILLE
PHILA. -> PHILADELPHIA
PHILADELPHILA -> PHILADELPHIA
PHILADEPHIA -> PHILADELPHIA
PHILDELPHIA -> PHILADELPHIA


References:
* https://cosmiccoding.com.au/tutorials/fuzzy_string_matching
* https://www.datacamp.com/community/tutorials/fuzzy-string-python